In [1]:
import pandas as pd

In [2]:
file_path1 = "Resources/schools_complete.csv"
file_path2 = "Resources/students_complete.csv"

schools_df = pd.read_csv(file_path1)
students_df = pd.read_csv(file_path2)

#schools_df.head()
#students_df.head()

In [3]:
schools_df.columns

Index(['School ID', 'school_name', 'type', 'size', 'budget'], dtype='object')

In [4]:
students_df.columns

Index(['Student ID', 'student_name', 'gender', 'grade', 'school_name',
       'reading_score', 'math_score'],
      dtype='object')

In [5]:
Total_Schools = len(schools_df)
Total_Students = len(students_df)
Total_Budget = schools_df["budget"].sum()
Average_Math_Score = students_df["math_score"].mean()
Average_Reading_Score = students_df["reading_score"].mean()
Percent_Passing_Math = students_df["math_score"][students_df["math_score"] >= 65].count() / Total_Students
Percent_Passing_Reading = students_df["reading_score"][students_df["reading_score"] >= 65].count() / Total_Students
Overall_Passing_Rate = (Percent_Passing_Math + Percent_Passing_Reading) / 2

In [6]:
District_Summary_df = pd.DataFrame({"Total Schools": [Total_Schools],
                                   "Total Students": [Total_Students],
                                   "Total Budget": [Total_Budget],
                                   "Average Math Score": [Average_Math_Score],
                                   "Average Reading Score": [Average_Reading_Score],
                                   "% Passing Math": [Percent_Passing_Math],
                                   "% Passing Reading": [Percent_Passing_Reading],
                                   "Overall Passing Rate": [Overall_Passing_Rate]})
District_Summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,0.847281,0.961986,0.904634


In [7]:
schools2_df = schools_df.copy()

del schools2_df["School ID"]

schools2_df.rename(columns={'school_name':'School Name', 'type': 'School Type', 'size': 'Total Students',
                                  'budget': 'Total School Budget'}, inplace=True)

schools2_df["Per Student Budget"] = schools2_df["Total School Budget"] / schools2_df["Total Students"]


students2_df = students_df.copy()

students2_df.rename(columns={'school_name': 'School Name'}, inplace=True)

students2_df = students2_df.groupby(["School Name"]).mean()

del students2_df["Student ID"]

students2_df.rename(columns={'math_score': 'Average Math Score',
                             'reading_score': 'Average Reading Score'}, inplace=True)


School_Summary_df = pd.merge(schools2_df, students2_df, on="School Name")

In [8]:
students3_df = students_df.copy()
students3_df.rename(columns={"school_name": "School Name"}, inplace=True)


Passing_math = students3_df["School Name"][students3_df["math_score"] >= 65].value_counts().rename_axis("School Name").reset_index(name = "Passingmath")

Passing_reading = students3_df["School Name"][students3_df["reading_score"] >= 65].value_counts().rename_axis("School Name").reset_index(name = "Passingreading")


School_Summary_df = pd.merge(School_Summary_df, Passing_math, on="School Name")

School_Summary_df["% Passing Math"] = School_Summary_df["Passingmath"] / School_Summary_df["Total Students"]



School_Summary_df = pd.merge(School_Summary_df, Passing_reading, on = "School Name")

School_Summary_df["% Passing Reading"] = School_Summary_df["Passingreading"] / School_Summary_df["Total Students"]

School_Summary_df.drop(columns=["Passingmath", "Passingreading"], inplace=True)



School_Summary_df["Overall Passing Rate"] = (School_Summary_df["% Passing Math"] + School_Summary_df["% Passing Reading"]) / 2

School_Summary_df = School_Summary_df.reindex(columns= ["School Name", "School Type", "Total Students",
                                                        "Total School Budget", "Per Student Budget", "Average Math Score",
                                                        "Average Reading Score", "% Passing Math", "% Passing Reading",
                                                        "Overall Passing Rate"])

School_Summary_df

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,0.777168,0.944806,0.860987
1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,0.771787,0.945405,0.858596
2,Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,1.000000,1.000000,1.000000
3,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,0.777346,0.946063,0.861704
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,1.000000,1.000000,1.000000
5,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,1.000000,1.000000,1.000000
6,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,1.000000,1.000000,1.000000
7,Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,0.779140,0.945539,0.862339
8,Holden High School,Charter,427,248087,581.0,83.803279,83.814988,1.000000,1.000000,1.000000
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,1.000000,1.000000,1.000000


In [9]:
Top_Performing_Schools_df = School_Summary_df.sort_values("Overall Passing Rate", ascending=False)
Top_Performing_Schools_df.iloc[0:5,:]

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
2,Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,1.0,1.0,1.0
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,1.0,1.0,1.0
5,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,1.0,1.0,1.0
6,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,1.0,1.0,1.0
8,Holden High School,Charter,427,248087,581.0,83.803279,83.814988,1.0,1.0,1.0


In [10]:
Bottom_Performing_Schools_df = School_Summary_df.sort_values("Overall Passing Rate", ascending=True)
Bottom_Performing_Schools_df.iloc[0:5,:]

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,0.771787,0.945405,0.858596
13,Ford High School,District,2739,1763916,644.0,77.102592,80.746258,0.782037,0.938664,0.860350
0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,0.777168,0.944806,0.860987
3,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,0.777346,0.946063,0.861704
12,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,0.779668,0.944760,0.862214


In [11]:
students4_df = students_df.copy()
students4_df = students4_df[["grade","school_name", "math_score"]]
students4_df.rename(columns={"school_name":"School Name"}, inplace=True)
students4_df = students4_df.loc[students4_df["grade"] == "9th",:]
del students4_df["grade"]
students4_df = students4_df.groupby(["School Name"]).mean()
students4_df.rename(columns={"math_score": "Average 9th Grade Math Score"}, inplace=True)

students5_df = students_df.copy()
students5_df = students5_df[["grade","school_name", "math_score"]]
students5_df.rename(columns={"school_name":"School Name"}, inplace=True)
students5_df = students5_df.loc[students5_df["grade"] == "10th",:]
del students5_df["grade"]
students5_df = students5_df.groupby(["School Name"]).mean()
students5_df.rename(columns={"math_score": "Average 10th Grade Math Score"}, inplace=True)

Math_Scores_by_Grade_df = pd.merge(students4_df, students5_df, on="School Name")

students6_df = students_df.copy()
students6_df = students6_df[["grade","school_name", "math_score"]]
students6_df.rename(columns={"school_name":"School Name"}, inplace=True)
students6_df = students6_df.loc[students6_df["grade"] == "11th",:]
del students6_df["grade"]
students6_df = students6_df.groupby(["School Name"]).mean()
students6_df.rename(columns={"math_score": "Average 11th Grade Math Score"}, inplace=True)

Math_Scores_by_Grade_df = pd.merge(Math_Scores_by_Grade_df, students6_df, on="School Name")

students7_df = students_df.copy()
students7_df = students7_df[["grade","school_name", "math_score"]]
students7_df.rename(columns={"school_name":"School Name"}, inplace=True)
students7_df = students7_df.loc[students7_df["grade"] == "12th",:]
del students7_df["grade"]
students7_df = students7_df.groupby(["School Name"]).mean()
students7_df.rename(columns={"math_score": "Average 12th Grade Math Score"}, inplace=True)

Math_Scores_by_Grade_df = pd.merge(Math_Scores_by_Grade_df, students7_df, on="School Name")
Math_Scores_by_Grade_df

,Average 9th Grade Math Score,Average 10th Grade Math Score,Average 11th Grade Math Score,Average 12th Grade Math Score
School Name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [12]:
students8_df = students_df.copy()
students8_df = students8_df[["grade", "school_name", "reading_score"]]
students8_df.rename(columns={"school_name": "School Name"}, inplace=True)
students8_df = students8_df.loc[students8_df["grade"] == "9th",:]
del students8_df["grade"]
students8_df = students8_df.groupby(["School Name"]).mean()
students8_df.rename(columns={"reading_score": "Average 9th Grade Reading Score"}, inplace=True)

students9_df = students_df.copy()
students9_df = students9_df[["grade", "school_name", "reading_score"]]
students9_df.rename(columns={"school_name": "School Name"}, inplace=True)
students9_df = students9_df.loc[students9_df["grade"] == "10th",:]
del students9_df["grade"]
students9_df = students9_df.groupby(["School Name"]).mean()
students9_df.rename(columns={"reading_score": "Average 10th Grade Reading Score"}, inplace=True)

Reading_Scores_by_Grade_df = pd.merge(students8_df, students9_df, on="School Name")

students10_df = students_df.copy()
students10_df = students10_df[["grade", "school_name", "reading_score"]]
students10_df.rename(columns={"school_name": "School Name"}, inplace=True)
students10_df = students10_df.loc[students10_df["grade"] == "11th",:]
del students10_df["grade"]
students10_df = students10_df.groupby(["School Name"]).mean()
students10_df.rename(columns={"reading_score": "Average 11th Grade Reading Score"}, inplace=True)

Reading_Scores_by_Grade_df = pd.merge(Reading_Scores_by_Grade_df, students10_df, on="School Name")

students11_df = students_df.copy()
students11_df = students11_df[["grade", "school_name", "reading_score"]]
students11_df.rename(columns={"school_name": "School Name"}, inplace=True)
students11_df = students11_df.loc[students11_df["grade"] == "12th",:]
del students11_df["grade"]
students11_df = students11_df.groupby(["School Name"]).mean()
students11_df.rename(columns={"reading_score": "Average 12th Grade Reading Score"}, inplace=True)

Reading_Scores_by_Grade_df = pd.merge(Reading_Scores_by_Grade_df, students11_df, on="School Name")
Reading_Scores_by_Grade_df

,Average 9th Grade Reading Score,Average 10th Grade Reading Score,Average 11th Grade Reading Score,Average 12th Grade Reading Score
School Name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [13]:
bins=[570, 594.9, 619.9, 644.9, 670]
group_names = ["low Spending", "mid-low Spending",
               "mid-high Spending", "high Spending"]


schools3_df = schools2_df.copy()

schools3_df["School Spending"] = pd.cut(schools3_df["Per Student Budget"], bins, labels=group_names,
                                              include_lowest=True)


total_students_df = schools3_df.copy()
total_students_df = total_students_df[["School Spending", "Total Students"]]
total_students_df = total_students_df.groupby(["School Spending"])
total_students_df = total_students_df.sum()

students12_df = students_df.copy()
students12_df.rename(columns={"school_name": "School Name"}, inplace=True)


Scores_by_School_Spending_df = pd.merge(schools3_df, students12_df, on="School Name")

Scores_by_School_Spending_df = Scores_by_School_Spending_df[["School Name", "School Spending",
                                                             "reading_score", "math_score"]]


Passing_math_2 = Scores_by_School_Spending_df["School Spending"][Scores_by_School_Spending_df["math_score"] >= 65].value_counts().rename_axis("School Spending").reset_index(name = "Passingmath_b")

Passing_reading_2 = Scores_by_School_Spending_df["School Spending"][Scores_by_School_Spending_df["reading_score"] >= 65].value_counts().rename_axis("School Spending").reset_index(name = "Passingreading_b")


Scores_by_School_Spending_df = Scores_by_School_Spending_df.groupby(["School Spending"]).mean()
Scores_by_School_Spending_df = Scores_by_School_Spending_df.rename(columns={"reading_score":"Average Reading Score",
                                                                            "math_score":"Average Math Score"})


Scores_by_School_Spending_df = pd.merge(total_students_df, Scores_by_School_Spending_df, on="School Spending")


Scores_by_School_Spending_df = pd.merge(Scores_by_School_Spending_df, Passing_math_2, on="School Spending")
Scores_by_School_Spending_df["% Passing Math"] = Scores_by_School_Spending_df["Passingmath_b"] / Scores_by_School_Spending_df["Total Students"]


Scores_by_School_Spending_df = pd.merge(Scores_by_School_Spending_df, Passing_reading_2, on="School Spending")
Scores_by_School_Spending_df["% Passing Reading"] = Scores_by_School_Spending_df["Passingreading_b"] / Scores_by_School_Spending_df["Total Students"]
Scores_by_School_Spending_df.drop(["Total Students", "Passingmath_b", "Passingreading_b"], axis=1, inplace=True)


Scores_by_School_Spending_df["Overall Passing Rate"] = (Scores_by_School_Spending_df["% Passing Math"] + Scores_by_School_Spending_df["% Passing Reading"]) / 2

Scores_by_School_Spending_df = Scores_by_School_Spending_df.reindex(columns=["School Spending", "Average Math Score",
                                                                             "Average Reading Score", "% Passing Math",
                                                                             "% Passing Reading", "Overall Passing Rate"])
Scores_by_School_Spending_df

,School Spending,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,low Spending,83.363065,83.964039,1.000000,1.000000,1.000000
1,mid-low Spending,83.529196,83.838414,1.000000,1.000000,1.000000
2,mid-high Spending,78.061635,81.434088,0.817010,0.954126,0.885568
3,high Spending,77.049297,81.005604,0.778202,0.945261,0.861732


In [14]:
bins_2 = [420, 1939.9, 3459.9, 4980]
group_names_2 = ["Small", "Medium", "Large"]


schools4_df = schools2_df.copy()

schools4_df["School Size"] = pd.cut(schools4_df["Total Students"], bins_2, labels=group_names_2,
                                    include_lowest=True)


total_students_2_df = schools4_df.copy()
total_students_2_df = total_students_2_df[["Total Students", "School Size"]]
total_students_2_df = total_students_2_df.groupby(["School Size"])
total_students_2_df = total_students_2_df.sum()


students13_df = students_df.copy()
students13_df.rename(columns={"school_name": "School Name"}, inplace=True)


Scores_by_School_Size_df = pd.merge(schools4_df, students13_df, on="School Name")

Scores_by_School_Size_df = Scores_by_School_Size_df[["School Name", "School Size",
                                                     "reading_score", "math_score"]]


Passing_math_3 = Scores_by_School_Size_df["School Size"][Scores_by_School_Size_df["math_score"] >= 65].value_counts().rename_axis("School Size").reset_index(name="Passingmath_c")

Passing_reading_3 = Scores_by_School_Size_df["School Size"][Scores_by_School_Size_df["reading_score"] >= 65].value_counts().rename_axis("School Size").reset_index(name="Passingreading_c")


Scores_by_School_Size_df = Scores_by_School_Size_df.groupby(["School Size"]).mean()
Scores_by_School_Size_df = Scores_by_School_Size_df.rename(columns={"reading_score": "Average Reading Score",
                                                                    "math_score": "Average Math Score"})


Scores_by_School_Size_df = pd.merge(total_students_2_df, Scores_by_School_Size_df, on="School Size")


Scores_by_School_Size_df = pd.merge(Scores_by_School_Size_df, Passing_math_3, on="School Size")
Scores_by_School_Size_df["% Passing Math"] = Scores_by_School_Size_df["Passingmath_c"] / Scores_by_School_Size_df["Total Students"]


Scores_by_School_Size_df = pd.merge(Scores_by_School_Size_df, Passing_reading_3, on="School Size")
Scores_by_School_Size_df["% Passing Reading"] = Scores_by_School_Size_df["Passingreading_c"] / Scores_by_School_Size_df["Total Students"]
Scores_by_School_Size_df.drop(["Total Students", "Passingmath_c", "Passingreading_c"], axis=1, inplace=True)


Scores_by_School_Size_df["Overall Passing Rate"] = (Scores_by_School_Size_df["% Passing Math"] + Scores_by_School_Size_df["% Passing Reading"]) / 2

Scores_by_School_Size_df = Scores_by_School_Size_df.reindex(columns=["School Size", "Average Math Score",
                                                                     "Average Reading Score", "% Passing Math",
                                                                     "% Passing Reading", "Overall Passing Rate"])
Scores_by_School_Size_df


,School Size,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Small,83.436586,83.882857,1.000000,1.000000,1.000000
1,Medium,78.164034,81.654758,0.823659,0.954996,0.889328
2,Large,77.070764,80.928365,0.778891,0.945621,0.862256


In [15]:
schools5_df = schools2_df.copy()


total_students_3_df = schools2_df.copy()
total_students_3_df = total_students_3_df[["School Type", "Total Students"]]
total_students_3_df = total_students_3_df.groupby(["School Type"])
total_students_3_df = total_students_3_df.sum()


students14_df = students_df.copy()
students14_df.rename(columns={"school_name": "School Name"}, inplace=True)


Scores_by_School_Type_df = pd.merge(schools5_df, students14_df, on="School Name")

Scores_by_School_Type_df = Scores_by_School_Type_df[["School Name", "School Type",
                                                     "reading_score", "math_score"]]


Passing_math_4 = Scores_by_School_Type_df["School Type"][Scores_by_School_Type_df["math_score"] >= 65].value_counts().rename_axis("School Type").reset_index(name="Passingmath_d")

Passing_reading_4 = Scores_by_School_Type_df["School Type"][Scores_by_School_Type_df["reading_score"] >= 65].value_counts().rename_axis("School Type").reset_index(name="Passingreading_d")


Scores_by_School_Type_df = Scores_by_School_Type_df.groupby("School Type").mean()
Scores_by_School_Type_df = Scores_by_School_Type_df.rename(columns={"reading_score": "Average Reading Score",
                                                                    "math_score": "Average Math Score"})


Scores_by_School_Type_df = pd.merge(total_students_3_df, Scores_by_School_Type_df, on="School Type")


Scores_by_School_Type_df = pd.merge(Scores_by_School_Type_df, Passing_math_4, on="School Type")
Scores_by_School_Type_df["% Passing Math"] = Scores_by_School_Type_df["Passingmath_d"] / Scores_by_School_Type_df["Total Students"]


Scores_by_School_Type_df = pd.merge(Scores_by_School_Type_df, Passing_reading_4, on="School Type")
Scores_by_School_Type_df["% Passing Reading"] = Scores_by_School_Type_df["Passingreading_d"] / Scores_by_School_Type_df["Total Students"]
Scores_by_School_Type_df.drop(["Total Students", "Passingmath_d", "Passingreading_d"], axis=1, inplace=True)


Scores_by_School_Type_df["Overall Passing Rate"] = (Scores_by_School_Type_df["% Passing Math"] + Scores_by_School_Type_df["% Passing Reading"]) / 2

Scores_by_School_Type_df = Scores_by_School_Type_df.reindex(columns=["School Type", "Average Math Score",
                                                                     "Average Reading Score", "% Passing Math",
                                                                     "% Passing Reading", "Overall Passing Rate"])
Scores_by_School_Type_df

,School Type,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Charter,83.406183,83.902821,1.000000,1.000000,1.000000
1,District,76.987026,80.962485,0.778247,0.944803,0.861525
